In [1]:
import pandas as pd
import plotly.express as px
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, TextVectorization, Embedding
from tensorflow.keras.models import Model

from sklearn.preprocessing import LabelEncoder

from typing import List, Tuple
from tqdm import tqdm

In [2]:
def data_processing(cocktail_data_path: str, recipe_data_path: str) -> Tuple[pd.DataFrame, LabelEncoder, LabelEncoder]:
    """
    Read the input data, and transform into the structure needed for training a model
    
    Inputs:
        data_path:
            A string path to the input Mr Boston Cocktail Dataset
        
    Outputs:
        data:
            A pandas dataframe with all of the needed columns for training
        ingredient_encoder:
            A label encoder to transform ingredient names to numbers and back
        measurement_encoder:
            A label encoder to transform measurement amounts to numbers and back
    """
    
    cocktail_df = pd.read_csv(cocktail_data_path)
    
    cocktail_ingredients = pd.concat([cocktail_df["ingredient-1"], cocktail_df["ingredient-2"], cocktail_df["ingredient-3"], cocktail_df["ingredient-4"], cocktail_df["ingredient-5"], cocktail_df["ingredient-6"]]).unique()

    ingredient_encoder = LabelEncoder()
    ingredient_encoder.fit(ingredients)
    
    df["ingredient-1"] = ingredient_encoder.transform(df["ingredient-1"])
    df["ingredient-2"] = ingredient_encoder.transform(df["ingredient-2"])
    df["ingredient-3"] = ingredient_encoder.transform(df["ingredient-3"])
    df["ingredient-4"] = ingredient_encoder.transform(df["ingredient-4"])
    df["ingredient-5"] = ingredient_encoder.transform(df["ingredient-5"])
    df["ingredient-6"] = ingredient_encoder.transform(df["ingredient-6"])
    
    measurements = pd.concat([df["measurement-1"], df["measurement-2"], df["measurement-3"], df["measurement-4"], df["measurement-5"], df["measurement-6"]]).unique()

    measurement_encoder = LabelEncoder()
    measurement_encoder.fit(measurements)
    
    df["measurement-1"] = measurement_encoder.transform(df["measurement-1"])
    df["measurement-2"] = measurement_encoder.transform(df["measurement-2"])
    df["measurement-3"] = measurement_encoder.transform(df["measurement-3"])
    df["measurement-4"] = measurement_encoder.transform(df["measurement-4"])
    df["measurement-5"] = measurement_encoder.transform(df["measurement-5"])
    df["measurement-6"] = measurement_encoder.transform(df["measurement-6"])
    
    pieces = []
    pieces.append(df[["name"]])

    for i in range(6):
        temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"ingredient-{i+1}"]), columns=[f"I{i+1} - {c}" for c in ingredient_encoder.classes_])
        pieces.append(temp)

        temp = pd.DataFrame(data=tf.keras.utils.to_categorical(df[f"measurement-{i+1}"]), columns=[f"M{i+1} - {c}" for c in measurement_encoder.classes_])
        pieces.append(temp)

    data = pd.concat(pieces, axis=1)
    
    return data, ingredient_encoder, measurement_encoder
    

In [3]:
def prepare_model(data: pd.DataFrame, ingredient_encoder, measurement_encoder) -> Model:
    """
    
    """
    
    textvec = TextVectorization(
        max_tokens=20000,
        output_sequence_length=5,
        pad_to_max_tokens=True
    )

    ##TODO:
    ##Instead of using the names to adapt, use a more general text source
    extra_vocab = list(pd.read_csv("RAW_recipes.csv")["description"])
    name_vocab = list(data["name"])
    textvec.adapt(name_vocab + extra_vocab)

    voc = textvec.get_vocabulary()
    print(f"Found {len(voc)} different words!")
    word_index = dict(zip(voc, range(len(voc))))
    
    path_to_glove_file = "glove.6B.300d.txt"

    embeddings_index = {}
    with open(path_to_glove_file, encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    
    num_tokens = len(voc) + 2
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    
    encoder_inputs = Input(shape=(1,), dtype=tf.string)

    e = textvec(encoder_inputs)

    e = Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        trainable=False,
    )(e)

    encoder_outputs = LSTM(128)(e)

    d = Dense(64)(encoder_outputs)
    d = Dropout(0.25)(d)
    d = Dense(64)(d)
    d = Dropout(0.25)(d)

    i1 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i1_output")(d)
    i2 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i2_output")(d)
    i3 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i3_output")(d)
    i4 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i4_output")(d)
    i5 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i5_output")(d)
    i6 = Dense(len(ingredient_encoder.classes_), activation="sigmoid", name="i6_output")(d)

    m1 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m1_output")(d)
    m2 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m2_output")(d)
    m3 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m3_output")(d)
    m4 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m4_output")(d)
    m5 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m5_output")(d)
    m6 = Dense(len(measurement_encoder.classes_), activation="sigmoid", name="m6_output")(d)

    decoder_outputs = [i1, m1, i2, m2, i3, m3, i4, m4, i5, m5, i6, m6]
    model = Model(encoder_inputs, decoder_outputs)
        
    losses = {
        "i1_output":"categorical_crossentropy",
        "i2_output":"categorical_crossentropy",
        "i3_output":"categorical_crossentropy",
        "i4_output":"categorical_crossentropy",
        "i5_output":"categorical_crossentropy",
        "i6_output":"categorical_crossentropy",
        "m1_output":"categorical_crossentropy",
        "m2_output":"categorical_crossentropy",
        "m3_output":"categorical_crossentropy",
        "m4_output":"categorical_crossentropy",
        "m5_output":"categorical_crossentropy",
        "m6_output":"categorical_crossentropy",
    }

    optimizer='adam'
    metrics = {
        "i1_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i2_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i3_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i4_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i5_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "i6_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m1_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m2_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m3_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m4_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m5_output":tf.metrics.CategoricalAccuracy(name='acc'),
        "m6_output":tf.metrics.CategoricalAccuracy(name='acc'),
    }

    model.compile(loss=losses, metrics=metrics, optimizer=optimizer)
    
    return model

In [4]:
def fit_model(model, data, ingredient_encoder, measurement_encoder, epochs: int = 100) -> None:
    """
    
    """
    
    X = data[["name"]].values

    Ys = {}

    for i in range(6):
        temp = data[[col for col in data.columns if f"I{i+1} - " in col]]
        Ys[f"i{i+1}_output"] = temp.values
        if temp.values.shape[1] != len(ingredient_encoder.classes_):
            raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")

        temp = data[[col for col in data.columns if f"M{i+1} - " in col]]
        Ys[f"m{i+1}_output"] = temp.values
        if temp.values.shape[1] != len(measurement_encoder.classes_):
            raise RuntimeError(f"Too many columns pulled! Problem on I{i+1}")

    model.fit(X, Ys, epochs=epochs, verbose=1)

In [32]:
def transform_str2list(input_string) -> List[str]:
    """
    Takes a string that contains a list like: "[1, 2, 3]" and turns it
    into a list of strings like: ["1", "2", "3"]
    """
    
    clean = input_string[1:-1]
    pieces = clean.split(",")
    
    output_list = [piece.strip().lower().replace("\'", "").replace('"', "") for piece in pieces]
    
    return output_list

In [5]:
def create_recipe(name: str, model: Model, ingredient_encoder: LabelEncoder, measurement_encoder: LabelEncoder) -> List[str]:
    """
    
    """
    
    predicted_ingredients = model.predict([name])
    
    recipe = []
    for i in range(6):
        ingredient = predicted_ingredients[2*i]
        measurement = predicted_ingredients[2*i + 1]
        
        ingredient = np.argmax(ingredient)
        measurement = np.argmax(measurement)
        
        ingredient = ingredient_encoder.classes_[ingredient]
        measurement = measurement_encoder.classes_[measurement]
        
        if str(ingredient).strip().lower() == "nan":
            continue
        else:
            recipe.append(f"{measurement} of {ingredient}")
    
    return recipe

In [6]:
#data, ingredient_encoder, measurement_encoder = data_processing("mr-boston-flattened.csv")
#model = prepare_model(data, ingredient_encoder, measurement_encoder)
#fit_model(model, data, ingredient_encoder, measurement_encoder)

In [7]:
#recipe = create_recipe("Tomato Soup", model, ingredient_encoder, measurement_encoder)
#print(recipe)

In [20]:
recipe_df = pd.read_csv("Data/RAW_recipes.csv")

recipe_ingredients = set()

for ingredient_list in tqdm(recipe_df["ingredients"]):
    temp_set = set(transform_str2list(ingredient_list))
    recipe_ingredients = recipe_ingredients.union(temp_set)

recipe_ingredients = sorted(list(recipe_ingredients))
recipe_ingredients = [i.replace("\'", "").replace('"', "") for i in recipe_ingredients]
recipe_ingredients

100%|████████████████████████████████████████████████████████████████████████| 231637/231637 [00:48<00:00, 4820.05it/s]


['adolphs meat tenderizer',
 'annies goddess dressing',
 'aunt janes krazy mixed up salt',
 'bakers angel flake sweetened coconut',
 'bakers germans chocolate',
 'bakers joy',
 'bakers semi-sweet baking chocolate',
 'bakers semi-sweet chocolate',
 'bakers special dry milk',
 'bakers unsweetened chocolate square',
 'bakers unsweetened chocolate squares',
 'bakers yeast',
 'bells seasoning',
 'better n peanut butter spread',
 'betty crockers hamburger helper',
 'birds custard mix',
 'birds eye custard',
 'bobs red mill gluten-free all-purpose baking flour',
 'boones farm strawberry hill wine',
 'breakstones sour cream',
 'brewers yeast',
 'bushs red beans',
 'butterfinger bbs',
 'campbells cheddar cheese soup',
 'campbells chicken gumbo soup',
 'campbells chicken soup',
 'campbells chunky beef with country vegetables soup',
 'campbells chunky ready to serve vegetable soup',
 'campbells condensed beef broth',
 'campbells condensed chicken broth',
 'campbells condensed cream of asparagus s

In [21]:
cocktail_df = pd.read_csv("Data/mr-boston-flattened.csv")
cocktail_ingredients = pd.concat([cocktail_df["ingredient-1"], cocktail_df["ingredient-2"], cocktail_df["ingredient-3"], cocktail_df["ingredient-4"], cocktail_df["ingredient-5"], cocktail_df["ingredient-6"]]).unique()
cocktail_ingredients = [str(i).strip().lower().replace(",", "") for i in cocktail_ingredients]
cocktail_ingredients

['light rum',
 'apple schnapps',
 'juice of a lime',
 'dark rum',
 'bourbon whiskey',
 'amaretto',
 'scotch whiskey',
 'bacardi rum',
 'brandy',
 'gin',
 'sloe gin',
 'sweet vermouth',
 'tanqueray gin',
 'lemon juice',
 'straight rye whiskey',
 'grenadine',
 'green chartreuse',
 'irish whiskey',
 'dry vermouth',
 'juice of a lemon',
 'orange',
 'absinthe',
 'rye or bourbon whiskey',
 'light vermouth',
 'anisette',
 'kummel',
 'bourbon or rye whiskey',
 'tennessee whiskey',
 'vodka',
 'grapefruit juice',
 'fresh rosemary sprig',
 'powdered sugar',
 'wide spiral of lemon zest',
 'lemon-flavored vodka',
 'lime juice',
 'blended scotch whiskey',
 'apricot flavored brandy',
 'bitters',
 'juice of orange',
 'orange bitters',
 'blended scotch whiskey',
 'white whiskey',
 'dry gin',
 'forbidden fruit',
 'applejack',
 'port',
 'apple brandy',
 'maraschino',
 'coffee-flavored brandy',
 'single-malt scotch whisky',
 'hennessy v.s cognac',
 'cherry-flavored brandy',
 'creme de menthe (white)',
 'c

In [75]:
sorted(cocktail_ingredients)

['100-proof vodka',
 '151-proof rum',
 '17-year-old j. wray and nephew ltd. rum',
 '7-up',
 'absinthe',
 'absinthe',
 'absinthe or pastis',
 'absinthe substitute',
 'acai berry flavored vodka',
 'african rum',
 'agave nectar',
 'agave nectar strawberry slice',
 'aged rhum agricole',
 'allspice liqueur (pimento dram)',
 'almond extract',
 'almond milk',
 'almond or orgeat syrup',
 'amaretto',
 'amaretto di saronno',
 'amaro',
 'amaro nonino',
 'amer picon or torani amer',
 'amontillado sherry',
 'anejo tequila',
 'anejo tequila',
 'angostura bitters',
 'angostura bitters',
 'angostura bitters flamed lemon twist',
 'angostura bitters lime wheel',
 'anis',
 'anisette',
 'aperol',
 'apple brandy',
 'apple cider',
 'apple flavored brandy',
 'apple juice',
 'apple schnapps',
 'apple schnapps grapefruit twist',
 'apple slice',
 'applejack',
 'applejack',
 'apricot flavored brandy',
 'apricot nectar',
 'apricot-flavored brandy',
 'aquavit',
 'aquavit or vodka',
 'armagnac',
 'aromatic bitters'

In [23]:
all_ingredients = list(set(recipe_ingredients).union(set(cocktail_ingredients)))
len(all_ingredients)

15329

In [24]:
len(cocktail_ingredients)

688

In [26]:
ingredient_encoder = LabelEncoder()
ingredient_encoder.fit(all_ingredients)
ingredient_encoder

(15329,)

In [59]:
recipe_inputs = [name.split(" ") for name in recipe_df["name"].astype(str).str.strip()]
recipe_inputs = [" ".join([word for word in recipe if word != ""]) for recipe in recipe_inputs]
recipe_outputs = list(recipe_df["ingredients"].apply(transform_str2list))
recipe_outputs = [ingredient_encoder.transform(recipe) for recipe in tqdm(recipe_outputs)]

recipe_outputs

  0%|▏                                                                          | 523/231637 [00:08<1:03:28, 60.68it/s]

KeyboardInterrupt: 

In [76]:
cocktail_inputs = [name.lower().split(" ") for name in cocktail_df["name"].astype(str).str.strip()]
cocktail_inputs = [" ".join([word for word in cocktail if word != ""]) for cocktail in cocktail_inputs]
cocktail_df["Ingredient_list"] = cocktail_df["ingredient-1"].astype(str).str.replace("NaN", "") + ","
cocktail_df["Ingredient_list"] += cocktail_df["ingredient-2"].astype(str).str.replace("NaN", "") + ","
cocktail_df["Ingredient_list"] += cocktail_df["ingredient-3"].astype(str).str.replace("NaN", "") + ","
cocktail_df["Ingredient_list"] += cocktail_df["ingredient-4"].astype(str).str.replace("NaN", "") + ","
cocktail_df["Ingredient_list"] += cocktail_df["ingredient-5"].astype(str).str.replace("NaN", "") + ","
cocktail_df["Ingredient_list"] += cocktail_df["ingredient-6"].astype(str).str.replace("NaN", "")
cocktail_df["Ingredient_list"] = cocktail_df["Ingredient_list"].str.replace(",,", ",")
cocktail_df["Ingredient_list"] = cocktail_df["Ingredient_list"].str.replace(",,", ",")
cocktail_df["Ingredient_list"] = cocktail_df["Ingredient_list"].str.replace(",,", ",")
cocktail_df["Ingredient_list"] = cocktail_df["Ingredient_list"].str.replace(",,", ",")
cocktail_df["Ingredient_list"] = cocktail_df["Ingredient_list"].str.replace(",,", ",")
cocktail_outputs = list(cocktail_df["Ingredient_list"].str.strip())
cocktail_outputs = [[ing.strip().lower() for ing in cocktail.split(",") if ing != ""] for cocktail in cocktail_outputs]
cocktail_outputs = [ingredient_encoder.transform(recipe) for recipe in tqdm(cocktail_outputs)]

cocktail_inputs



  0%|                                                                                          | 0/990 [00:00<?, ?it/s]

  1%|▍                                                                                 | 6/990 [00:00<00:16, 58.82it/s]

  1%|▉                                                                                | 12/990 [00:00<00:16, 58.82it/s]

  2%|█▍                                                                               | 18/990 [00:00<00:16, 59.22it/s]

  2%|█▉                                                                               | 24/990 [00:00<00:16, 59.06it/s]

  3%|██▍                                                                              | 30/990 [00:00<00:16, 58.36it/s]

  4%|██▉                                                                              | 36/990 [00:00<00:16, 57.65it/s]

  4%|███▍                                                                             | 42/990 [00:00<00:16, 57.49it/s]

  5%|███▉                     

ValueError: y contains previously unseen labels: 'fresh raspberries and strawberries'



  5%|███▉                                                                             | 48/990 [00:20<00:16, 57.82it/s]

[['light rum', 'passion fruit syrup', 'lemon juice', 'lime juice'],
 ['light rum', 'sweet vermouth', 'juice of orange', 'juice of a lime'],
 ['apple schnapps', 'cinnamon schnapps', 'apple slice'],
 ['juice of a lime', 'powdered sugar', 'light rum'],
 ['dark rum',
  'cranberry juice',
  'pineapple juice',
  'orange curacao',
  'sour mix'],
 ['bourbon whiskey',
  'fresh lemon juice',
  'simple syrup',
  'soda water',
  'orange and lemon wheels',
  'maraschino cherry'],
 ['light rum', 'cherry-flavored brandy', 'light cream'],
 ['light rum', 'lime juice', 'triple sec', 'maraschino'],
 ['light rum', 'creme de baa', 'chilled champagne'],
 ['amaretto', 'fresh lemon juice', 'simple syrup', 'soda water'],
 ['scotch whiskey',
  'fresh carrot juice',
  'tawny port',
  'fresh lemon juice',
  'agave nectar'],
 ['bacardi rum', 'juice of a lime', 'grenadine'],
 ['light rum', 'sweet vermouth', 'apple brandy', 'grenadine', 'lemon juice'],
 ['brandy', 'gin', 'anisette', 'sweet vermouth', 'grenadine'],
 

In [68]:
nametextvec = TextVectorization(
    max_tokens=20000,
    output_sequence_length=10,
    pad_to_max_tokens=True
)

description_vocab = list(recipe_df["description"])
name_vocab = list(recipe_inputs) + list(cocktail_inputs)
full_vocab = name_vocab + description_vocab

nametextvec.adapt(full_vocab)
name_voc = nametextvec.get_vocabulary()

KeyboardInterrupt: 

In [ ]:
ingtextvec = TextVectorization(
    max_tokens=15000,
    output_sequence_length=40,
    pad_to_max_tokens=True
)

vocab = list(recipe_outputs) + list(cocktail_outputs)

ingtextvec.adapt(vocab)
ing_voc = ingtextvec.get_vocabulary()

In [55]:
def create_embedding_matrix(voc):
    
    word_index = dict(zip(voc, range(len(voc))))
    path_to_glove_file = "glove.6B.300d.txt"

    embeddings_index = {}
    with open(path_to_glove_file, encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs

    num_tokens = len(voc) + 2
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    
    return embedding_matrix

In [54]:
encoder_inputs = Input(shape=(1,), dtype=tf.string)

e = nametextvec(encoder_inputs)

e = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)(e)

_, encoder_h, encoder_c = LSTM(128, return_state=True)(e)

In [ ]:
decoder_inputs = Input(shape=(1,), dtype=tf.string)

In [43]:
cocktail_df

,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,instructions,glass,glass-size
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Combine ingredients with a cup of crushed ice ...,Old-Fashioned Glass,6 to 8 ounces
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Juice of a Lime,NaN,NaN,NaN,NaN,Shake with ice and strain into old-fashioned g...,Old-Fashioned Glass,6 to 8 ounces
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,Apple slice,NaN,NaN,NaN,NaN,NaN,NaN,Pour into ice-filled old-fashioned glass. Garn...,Old-Fashioned Glass,6 to 8 ounces
3,Cuban Cocktail No. 1,Cocktail Classics,1/2 oz,Juice of a Lime,1/2 oz,Powdered Sugar,2 oz,Light Rum,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6 or more ounces
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Mix all ingredients except curacao with ice, s...",Collins Glass,14 to 16 ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Wallis Blue Cocktail,Gin,1,"Lime wedge, superfine sugar",1 oz,Gin,1 oz,Triple Sec,1 oz,Fresh Lime Juice,NaN,NaN,NaN,NaN,Rim old-fashioned glass with lime and sugar. F...,Old-Fashioned Glass,6 to 8 ounces
986,Minnehaha Cocktail,Cocktail Classics,1/4 oz,Juice of Orange,1/2 oz,Dry Vermouth,1/2 oz,Sweet Vermouth,1 oz,Old Mr. Boston Dry Gin,NaN,NaN,NaN,NaN,Shake well with cracked ice and strain into 4 ...,Cocktail Glass,6 or more ounces
987,Wallick Cocktail,Gin,1 1/2 oz,Gin,1 1/2 oz,Dry Vermouth,1 oz,Triple Sec,NaN,NaN,NaN,NaN,NaN,NaN,Stir with ice and strain into chilled cocktail...,Cocktail Glass,6 or more ounces
988,Waikiki Beachcomber,Gin,3/4 oz,Gin,3/4 oz,Triple Sec,1/2 oz,Pineapple Juice,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into chilled cocktai...,Cocktail Glass,6 or more ounces
